# 0010_fastcore_meta_summary

## import

In [ ]:
from fastdebug.utils import *
from fastdebug.core import *

## fastcore

In [ ]:
import fastcore

In [ ]:
whichversion("fastcore")

fastcore: 1.5.27 
Python supercharged for fastai development    
Jeremy Howard and Sylvain Gugger 
https://github.com/fastai/fastcore/     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore


In [ ]:
whatinside(fastcore, lib=True)

The library has 21 modules
['_modidx',
 '_nbdev',
 'all',
 'basics',
 'dispatch',
 'docments',
 'docscrape',
 'foundation',
 'imports',
 'meta',
 'nb_imports',
 'net',
 'parallel',
 'script',
 'shutil',
 'style',
 'test',
 'transform',
 'utils',
 'xdg',
 'xtras']


In [ ]:
from fastcore.meta import *
import fastcore.meta as fm

In [ ]:
whatinside(fm, dun=True)

fastcore.meta has: 
13 items in its __all__, and 
43 user defined functions, 
19 classes or class objects, 
2 builtin funcs and methods, and
74 callables.

test_sig:            function    (f, b)
FixSigMeta:          metaclass, type    (name, bases, dict)
PrePostInitMeta:     metaclass, type    (name, bases, dict)
AutoInit:            class, PrePostInitMeta    (*args, **kwargs)
NewChkMeta:          metaclass, type    (name, bases, dict)
BypassNewMeta:       metaclass, type    (name, bases, dict)
empty2none:          function    (p)
anno_dict:           function    (f)
use_kwargs_dict:     decorator, function    (keep=False, **kwargs)
use_kwargs:          decorator, function    (names, keep=False)
delegates:           decorator, function    (to: function = None, keep=False, but: list = None)
method:              function    (f)
funcs_kwargs:        decorator, function    (as_method=False)


## Review individual funcs and classes

In [ ]:
fastlist()

test_sig.py
BypassNewMeta.py
FixSigMeta.py
funcs_kwargs.py
NewChkMeta.py
AutoInit.py
method.py
_rm_self.py
delegates.py
PrePostInitMeta.py
_funcs_kwargs.py
whatinside.py


In [ ]:
fastview(NewChkMeta)

class NewChkMeta(FixSigMeta):=============================================================(0)       
    "Metaclass to avoid recreating object passed to constructor"==========================(1) # NewChkMeta is a metaclass inherited from FixSigMea; it makes its own __call__; when its class instance, e.g., _T, create object instances (e.g, t) without args nor kwargs but only x, and x is an object of the instance class, then return x; otherwise, create and return a new object created by the instance class's super class' __call__ method with x as param; In other words, t = _T(3) will create a new obj; _T(t) will return t; _T(t, 1) or _T(t, b=1) will also return a new obj; 
    def __call__(cls, x=None, *args, **kwargs):===========================================(2) # how to create a __call__ method with param cls, x, *args, **kwargs;; 
        if not args and not kwargs and x is not None and isinstance(x,cls): return x======(3) # how to express no args and no kwargs and x is an instance o

In [ ]:
fastview(FixSigMeta)

class FixSigMeta(type):===================================================================(0)       
    "A metaclass that fixes the signature on classes that override `__new__`"=============(1) # Any class having FixSigMeta as metaclass will have its own __init__ func stored in its attr __signature__;FixSigMeta uses its __new__ to create a class instance; then check whether its class instance has its own __init__;if so, remove self from the sig of __init__; then assign this new sig to __signature__ for the class instance;; 
    def __new__(cls, name, bases, dict):==================================================(2) # how does a metaclass create a class instance; what does super().__new__() do here;; 
        res = super().__new__(cls, name, bases, dict)=====================================(3)       
        if res.__init__ is not object.__init__: res.__signature__ = _rm_self(inspect.signature(res.__init__)) # how to remove self from a signature; how to check whether a class' __init__

In [ ]:
fastsearch("how to get signature's parameters")

Found a line: ========>
        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items() # How to access a signature's parameters as a dict?; How to replace the kind of a parameter with a different kind?; how to check whether a parameter has a default value?; How to check whether a string is in a dict and a list?; how dict.items() and dict.values() differ;  (14)
The source where the line is from ==========>
def delegates(to:FunctionType=None, # Delegatee===========================================(0) # how to make delegates(to) to have to as FunctionType and default as None; 
              keep=False, # Keep `kwargs` in decorated function?==========================(1)       
              but:list=None): # Exclude these parameters from signature===================(2) # how to make delegates(to, but) to have 'but' as list and default as None; 
    "Decorator: replace `**kwargs` in signature with params from `to`"====================(3) 

## What is fastcore.meta all about